In [ ]:
import RF_input_parameters as RF_inp
import RF_tile_creator as RF_tc

RF_params = RF_inp.RF_Parameters()  # Create base parameters
RF_tc.create_tiles(RF_params)


In [2]:
if RF_params.add_classifier_params().model_path is None:
    import RF_trainer as RF_tr
    RF_tr.train_model(RF_params)

In [ ]:
import RF_tile_classifier as RF_cl
RF_cl.classify_tiles(RF_params)

In [ ]:
import os
import RF_input_parameters as RF_inp
import RF_tile_creator as RF_tc

#check DEM and Ortho path location
inputs_dict =   {'UM2_071922': [r"Y:\ATD\GIS\East_Troublesome\RF Vegetation Filtering\Input DEMs and Orthos\UM2_2023____071922_PostError_PCFiltered_Ortho_5cm.tif",
                                r"Y:\ATD\GIS\East_Troublesome\RF Vegetation Filtering\Input DEMs and Orthos\UM2_2023____071922_PostError_PCFiltered_DEM_5cm.tif"],
                 'UM1_090822' : [r"Y:\ATD\GIS\East_Troublesome\RF Vegetation Filtering\Input DEMs and Orthos\UM1_2023____090822_PostError_PCFiltered_Ortho_5cm.tif",
                                r"Y:\ATD\GIS\East_Troublesome\RF Vegetation Filtering\Input DEMs and Orthos\UM1_2023____090822_PostError_PCFiltered_DEM_5cm.tif"]
}


for key, value in inputs_dict.items():
    RF_params = RF_inp.RF_Parameters()  # Create base parameters
    RF_params.DEM_path = value[1]
    RF_params.ortho_path = value[0]
    RF_params.output_folder = r"Y:\ATD\GIS\East_Troublesome\RF Vegetation Filtering\{}".format(key)
    print(RF_params.output_folder)  
    if not os.path.exists(RF_params.output_folder):
        os.makedirs(RF_params.output_folder)
    RF_params.record_params()
    RF_tc.create_tiles(RF_params)
    import RF_tile_classifier as RF_cl
    RF_cl.classify_tiles(RF_params)

